## Chapter 2: Large-scale data analysis with Spacy (by. huffon)

- 이번 챕터에서는 큰 용량의 데이터에서 원하는 정보를 추출하는 방법을 배워볼 것입니다.
- 해당 튜토리얼을 통해 `spaCy` 라이브러리의 자료구조들을 생성하는 법과, 
- 어떻게 하면 효과적으로 통계 기반과 규칙 기반 분석 방법을 결합할 수 있을지에 대하여 학습하게 될 것입니다. 

### 1. Data Structures (1): Vocab, Lexemes and StringStore
#### Shared vocab and string store (1)
- `Vocab`: doc 객체들 간 공유하는 단어를 저장하는 자료구조
- 메모리를 절약하기 위해, spaCy는 모든 문자열을 해시 값으로 인코드
- 문자열은 `nip.vocab.strings`를 통해 `StringStore`에 단 한 번 저장됨

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp("I love coffee")
print('hash value: ', nlp.vocab.strings['coffee'])
print('string value: ', nlp.vocab.strings[3197928453018144401])

In [ ]:
doc = nlp("I love coffee")
print('hash value: ', doc.vocab.strings['coffee'])

- `lexeme`는 vocab의 최소 단위 객체로 문맥에 독립적인 단어 정보들을 포함
    - 단어 정보의 예: `lexeme.text`와 `lexeme.orth` (해시 값) 
    - `lexeme.is_alpha`와 같은 어휘 속성 역시 포함

<br/>
<img src='https://course.spacy.io/vocab_stringstore.png' width=500>

In [ ]:
doc = nlp("I love coffee")
lexeme = nlp.vocab['coffee']

print('{} / {} / {}'.format(lexeme.text, lexeme.orth, lexeme.is_alpha))

### 2. Strings to hashes

- 해시 값을 얻기 위해 문자열 **PERSON**을 `nlp.vocab.strings`에 대입해봅시다.
- 문자열을 얻기 위해 **PERSON**의 해시 값을 다시 대입해봅시다.

In [ ]:
doc = nlp("David Bowie is a PERSON")

person_hash = nlp.vocab.strings["PERSON"]
print(person_hash)

person_string = nlp.vocab.strings[person_hash]
print(person_string)

### 3. Data Structures (2): Doc, Span and Token

### The Doc object

In [ ]:
from spacy.lang.en import English
nlp = English()

# Doc 객체 임포트
from spacy.tokens import Doc

# 단어 리스트와 공백 리스트 정의
words = ['Hello', 'world', '!']
spaces = [True, False, False]

# 앞서 정의한 리스트들을 이용해 doc 객체 수동으로 생성
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc)

### The Span object

<img src='https://course.spacy.io/span_indices.png' width=500>

In [ ]:
# Doc과 Span 클래스 임포트
from spacy.tokens import Doc, Span

# 단어 리스트와 공백 리스트 정의
words = ['Hello', 'world', '!']
spaces = [True, False, False]

# 위에서 했던 것과 같이 Doc 객체 수동으로 정의
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Doc 객체 이용해 Span 객체를 수동으로 정의
span = Span(doc, 0, 2)
print(span)

# Span 객체에 라벨 부여
span_with_label = Span(doc, 0, 2, label="Greeting")

# doc 객체의 엔티티로 생성한 Span 추가
doc.ents = [span_with_label]

- `Doc`과 `Span` 은 매우 강력한 객체입니다.
    - 해당 객체들로 오래오래 사용하고, 최대한 나중에 string으로 변환합시다 !
- Token 객체의 속성을 최대한 활용하세요 ! (e.g. token.i를 사용해 index 추출 가능)

- Doc 객체에서 사용된 단어들의 저장을 위해 vocab을 Doc에 넘겨주는 것을 잊지마세요 !

In [ ]:
import spacy
from spacy.tokens import Doc

nlp = spacy.load("en_core_web_sm")

# 목표 텍스트: "Go, get started!"
words = ["Go", ",", "get", "started", "!"]
spaces = [False, True, True, False, False]

# 단어, 공백 리스트 + vocab 이용해 Doc 객체 수동 생성
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

- "David Bowie"를 나타내는 `span` 객체를 생성 후, `"PERSON"` 라벨을 부여하세요
- "David Bowie" 를 나타내는 `span` 객체를 doc.ents에 덮어쓰세요

In [ ]:
from spacy.lang.en import English
from spacy.tokens import Doc, Span

nlp = English()

words = ["I", "like", "David", "Bowie"]
spaces = [True, True, True, False]

# 단어, 공백 리스트 + vocab 이용해 Doc 객체 수동 생성
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

# "David Bowie"를 커버하는 Span 객체를 doc으로부터 생성 후, "PERSON" 라벨 부여
span = Span(doc, 2, 4, label="PERSON")
print(span.text, '/', span.label_)

# Span 객체를 doc 엔티티에 추가
doc.ents = [span]

# 엔티티의 텍스트와 라벨 속성을 출력
print([(ent.text, ent.label_) for ent in doc.ents])

### 4. Word vectors and semantic similarities

- `spaCy`는 두 객체를 비교해 **유사도** 예측 가능
- `Doc.similarity()`, `Span.similarity()`, `Token.similarity()`
- 위 메소드들은 인자로 다른 객체를 받으며, `0-1` 사이의 유사도를 반환
- **주의**: word vector를 포함하고 있는 크기의 모델을 다운로드 받아야 함
    - `en_core_web_md` 혹은 `en_core_web_lg`
    - Small model은 word vector 포함하지 않음 !

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
# Word vector를 포함하는 크기의 모델 로드
nlp = spacy.load('en_core_web_md')

# 두 개의 객체를 비교해 유사도 추출
doc1 = nlp("I like fast food")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

In [ ]:
# pizza와 pasta 토큰 비교
doc = nlp("I like pizza and pasta")
token1 = doc[2]
token2 = doc[4]
print(token1.similarity(token2))

Token, Doc, Span 등 서로 다른 객체 간 비교도 가능

In [ ]:
# Doc과 Token의 비교
doc = nlp("I like pizza")
token = nlp("soap")[0]

print(doc.similarity(token))

In [ ]:
# Span과 Doc의 비교
span = nlp("I like pizza and pasta")[2:5]
doc = nlp("McDonalds sells burgers")

print(span.similarity(doc))

In [ ]:
doc = nlp("This was a great restaurant. Afterwards, we went to a really nice bar.")

# "great restaurant" Span과 "really nice bar" Span 정의
span1 = doc[3:5]
span2 = doc[12:15]
print(span1, '/', span2)

# Span 간 유사도 계산
similarity = span1.similarity(span2)
print(similarity)

#### spaCy는 어떻게 유사도를 측정하고 있을까요?
- 유사도는 **Word vectors**를 이용해 결정됩니다.
- 이는 단어들의 의미를 다차원 공간에 나타낸 수치입니다.
- Word vector는 **Word2Vec**을 비롯한 다양한 알고리즘을 이용해 생성할 수 있습니다.
- 또한 외부 알고리즘을 통해 구한 유사도를 spaCy의 통계 모델에 추가할 수도 있습니다.
- spaCy에서 유사도를 구하는 기본 척도는 **코사인 유사도**이지만, 이는 변경이 가능합니다.
- `Doc`과 `Span` 객체의 유사도는 토큰 유사도의 평균으로 정의됩니다.
- 긴 문장보다 작은 문장의 유사도가 더 정확히 구해지는데, 이는 문장이 길어질수록 유사도를 구하는데 방해가 되는 단어들이 많아지기 때문입니다.

In [ ]:
doc = nlp("I have a banana")
# token.vector 속성 통해 word vector 출력 가능
print(doc[3].vector)

#### 유사도의 정의는 사용되는 어플리케이션에 따라 다를 수 있음
- 문맥과 어플리케이션의 목적에 따라 유사도의 정의가 달라져야 함 !

In [ ]:
doc1 = nlp("I like cats")
doc2 = nlp("I hate cats")

print(doc1.similarity(doc2))
# Is it similar or not?

### 5. Combining models and rules
- 통계 모델은 일반화 된 분석 결과를 반환해주지만, 모든 경우를 포함할 수는 없기에 규칙 기반의 시스템과 결합되어 사용되는 것이 바람직함
- `spaCy`에서는 `Matcher`, `PhraseMatcher` 클래스 등을 이용해 외부 규칙들을 더해줄 수 있음
- Matcher 클래스 복습

In [ ]:
from spacy.matcher import Matcher
from spacy import displacy

matcher = Matcher(nlp.vocab)

matcher.add('DOG', None, [{'LOWER': 'golden'}, {'LOWER': 'retriever'}])
doc = nlp("I have a Golden Retriever")

displacy.serve(doc, style="dep")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print('Matched span:', span.text)
    # Get the span's root token and root head token
    print('Root token:', span.root.text)
    print('Root head token:', span.root.head.text)

#### 효과적인 구(Phrase) 매칭 방법
- `PhraseMatcher`를 사용하면 보다 효율적으로 **구**를 매칭할 수 있음
- 이때 `Doc` 객체를 패턴으로 취함
- `Matcher` 클래스보다 빠르고 효과적 !
    - 때로는 개별 토큰을 추출하는 패턴을 작성하는 것보다 문자열 그대로를 매칭하는  `PhraseMatcher`을 사용하는 것이 더 효과적일 때가 있음
        - e.g.) 국가명과 같은 유한한 범주에서 문자열을 추출하고자 할 때

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("Golden Retriever") # Doc -> pattern
matcher.add('DOG', None, pattern)
doc = nlp("I have a Golden Retriever")

# 매칭 결과를 순회
for match_id, start, end in matcher(doc):
    # 매칭 결과를 Span 객체로 생성
    span = doc[start:end]
    print('Matched span:', span.text)